In [1]:
"""
Created on Mon Dec  6 16:29:49 2021

@author: meraidandouch
"""
import numpy as np 

import scipy.stats as sps

def read_data(file_name):
    """
    This file reads the fasta file with all rbd sequences
    
    file_name: rbd.fasta, a fasta file containing aa sequences of rbd sites in the COVID family 
    
    return data: a dictionary, accesion number ids with associated aa sequence values 
    """
    data = {}
    with open(file_name, 'rt') as f: 
        read_data = f.readline().strip('\n')
        while read_data != '':
            if read_data[0] == '>': 
                f_id = read_data
            else: 
                if f_id not in data.keys():
                    data[f_id] = read_data
                else: 
                    data[f_id] += read_data
            read_data = f.readline().strip('\n')
    return data
def count_aa(data): 
    """
    This functions concatenates all sequences counts the number of amino acids
    
    data: a dictionary, accesion number ids with associated aa sequence values 
    
    returns aa_dict, aa ids with associated sequence values  
    returns master_count, total number of aa in all sequences 
    """
    master_seq = ""
    master_count = 0
    aa = "QEFLIPGATCNURYSWKMBDHVX"
    aa_dict = {}
    for keys, values in data.items():
        master_seq += values
    for char in master_seq:
        if char in aa: 
            master_count += 1
            if char not in aa_dict.keys():
                aa_dict[char] = 1
            else:
                aa_dict[char] += 1
    return aa_dict, master_count

def freq_aa(aa_dict, master_count):
    """
    The function takes the frequency of each amino acid by dividing the number 
    of aa by the total aa
    
    aa_dict: aa ids with associated sequence values  
    master_count:  total number of aa in all sequences 
    
    returns freq_dict, aa as keys and freq as associated values 
    """
    mc = master_count
    freq_dict = {}
    a = 0 
    for keys, values in aa_dict.items(): 
            freq_dict[keys] = values/mc
            a += (values/mc)
    return freq_dict
      
class base_walker: 
    def __init__(self):
        """
        this class is the parent class base_walker 
        and generates aa sequences per random_walker object 
        """
        pass
    def one_step(self):
        pass
    def steps(self, seq):
        """
        seq : aa sequence from previous iteration 
          
        return none
        """
        for char in seq:
            self.one_step(char)
            
# one solution:
class random_walker(base_walker):
    def __init__(self, freq_dict):
        """
        this class inherits the abstract parent class base_walker 
        and generates aa per random_walker object 
        """
        # define stored variables, initial data
        self.x = "" #step, assumes that x = 0 at the start of the random walk.
        self.walk = self.x #list of random steps in a walk 
        self.prob = freq_dict 
        
    def one_step(self, aa):
        """
        this function determines whether a substitution events will 
        occur and if it does it generates a random aa else returns 
        the original aa, it then appends the aa to the remaining current
        sequence
        
        Return None
        """
        temp_dict = self.prob #store temp dictionary, keep original unchanged
      
        new_keys = list(temp_dict.keys()) #store keys
        new_values = list(temp_dict.values())#store values
        
        event = np.random.choice(['s', 'c'],p=[0.24,0.76]) #(Shah et al. 2020) mutation prob
        if event == 's' and aa != "[YRNPFEDIS]": #if a substituition event
            if aa == 'Y': #(Jia and Gong 2021) mutation prob
                aa = 'F'
            elif aa == 'S':#(Jia and Gong 2021) mutation prob
                aa = 'N'
            else: 
                self.x = np.random.choice(new_keys, p=new_values) #use freq of aa as probabilties 
        elif event == 'c': #if no substitution event, then return same aa 
            self.x = aa
            
        self.walk += self.x
    def get_walk(self):
        """
        Returns a list of the positions(scoring) of one walk 
        """
        # returns the history of the position of the walker
        return self.walk     

In [2]:
file_name = 'rbm (1).fasta'
data = read_data(file_name)
aa_dict, mc = count_aa(data)
freq_dict = freq_aa(aa_dict, mc)
total_seqs = [] 
total_dict = {}

my_walker = random_walker(freq_dict)
my_walker.steps(data['>UFP04971.1 surface glycoprotein [omicron variant Severe acute respiratory syndrome coronavirus 2]'])
new_walk = my_walker.get_walk()

print('>UFP04971.1[OMICRON]:',data['>UFP04971.1 surface glycoprotein [omicron variant Severe acute respiratory syndrome coronavirus 2]'])
print()
print("New Mutated Sequence: ", new_walk)
    

>UFP04971.1[OMICRON]: NKLDSKVSGNYNYLYRLFRKSNLKPFERDISTEIYQAGNKPCNGVAGFNCYFPLRSYSFRPTYGVGHQPY

New Mutated Sequence:  MKLFSKVSGNYNNSSRLFVEEYLKPFERDISTWIYQAGNNPCNGVAGENCCFYLRSSSFRPTYTKGHQPY
